# GCN 모델을 이용한 Graph Data training


In [1]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 10.0 MB/s eta 0:00:00


### 1) 필요한 라이브러리 선언

In [2]:
import time
import torch
import torch_geometric.transforms as T
import torch.nn as nn
from torch_geometric.nn import GCNConv
import torch.nn.functional as F

### 2) 현재 시간을 기록하여 시간 측정 시작



In [3]:
start_time = time.time()

### 3) 장치를 "cpu"로 설정

In [4]:
device = "cpu"

### 4) Dataset 준비 (Amazon)

In [5]:
from torch_geometric.datasets import Amazon

dataset = Amazon(root='./Amazon_data', name='computers')
graph = dataset[0]
split = T.RandomNodeSplit(num_val=0.1, num_test=0.2)
graph = split(graph)

Processing...
Done!


### 5) 모델 구축: GCN

In [6]:
class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(dataset.num_node_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        output = self.conv2(x, edge_index)
        return output

### 6) GCN 모델을 학습하는 함수 정의
입력으로는 모델, 그래프 데이터, 옵티마이저, 손실함수, 에폭 횟수가 주어짐

In [7]:
def train_node_classifier(model, graph, optimizer, criterion, n_epochs=200):
    # 에폭 횟수만큼 학습 반복
    for epoch in range(1, n_epochs + 1):
        model.train() # 모델을 학습 상태로 전환
        optimizer.zero_grad() # 그래디언트 초기화
        out = model(graph) # out :예측값
        loss = criterion(out[graph.train_mask], graph.y[graph.train_mask]) # loss 계산
        loss.backward() # Backpropagation 수행
        optimizer.step() # 파라미터 업데이트

        pred = out.argmax(dim=1) #out에서 가장 높은 값을 가지는 인덱스를 예측값으로 사용
        acc = eval_node_classifier(model, graph, graph.val_mask) # 모델 성능 검증

        if epoch % 10 == 0:
            print(f'Epoch: {epoch:03d}, Train Loss: {loss:.3f}, Val Acc: {acc:.3f}')

    return model

### 7) 노드 분류 모델의 성능을 평가하는 함수 정의
입력으로는 모델, 그래프, 마스크가 주어짐

In [8]:
def eval_node_classifier(model, graph, mask):

    model.eval() # 모델을 평가 모드로 전환
    # 모델의 출력 계산 -> argmax함수를 사용하여 출력 텐서에서 각 노드의 예측 클래스를 결정
    pred = model(graph).argmax(dim=1)
    # 예측된 클래스와 그래프의 실제 클래스를 비교하여 정확하게 분류된 노드의 수 계산
    correct = (pred[mask] == graph.y[mask]).sum()
    #print(pred[mask] + graph.y[mask])

    # 정확도 계산
    acc = int(correct) / int(mask.sum())

    return acc

### 8) 모델 훈련

In [9]:
gcn = GCN().to(device)
optimizer_gcn = torch.optim.Adam(gcn.parameters(), lr=0.01, weight_decay=5e-4)
criterion = nn.CrossEntropyLoss()
gcn = train_node_classifier(gcn, graph, optimizer_gcn, criterion)

Epoch: 010, Train Loss: 1.927, Val Acc: 0.436
Epoch: 020, Train Loss: 1.704, Val Acc: 0.463
Epoch: 030, Train Loss: 1.535, Val Acc: 0.492
Epoch: 040, Train Loss: 1.402, Val Acc: 0.500
Epoch: 050, Train Loss: 1.213, Val Acc: 0.711
Epoch: 060, Train Loss: 0.986, Val Acc: 0.742
Epoch: 070, Train Loss: 0.819, Val Acc: 0.750
Epoch: 080, Train Loss: 0.696, Val Acc: 0.812
Epoch: 090, Train Loss: 0.601, Val Acc: 0.841
Epoch: 100, Train Loss: 0.539, Val Acc: 0.851
Epoch: 110, Train Loss: 0.498, Val Acc: 0.856
Epoch: 120, Train Loss: 0.467, Val Acc: 0.863
Epoch: 130, Train Loss: 0.444, Val Acc: 0.868
Epoch: 140, Train Loss: 0.426, Val Acc: 0.870
Epoch: 150, Train Loss: 0.410, Val Acc: 0.873
Epoch: 160, Train Loss: 0.399, Val Acc: 0.875
Epoch: 170, Train Loss: 0.388, Val Acc: 0.876
Epoch: 180, Train Loss: 0.381, Val Acc: 0.876
Epoch: 190, Train Loss: 0.371, Val Acc: 0.881
Epoch: 200, Train Loss: 0.365, Val Acc: 0.878


### 9) 결과 분석 및 소요시간 측정

In [10]:
test_acc = eval_node_classifier(gcn, graph, graph.test_mask)
print(f'Test Acc: {test_acc:.3f}')

Test Acc: 0.893


In [11]:
# 시간 측정 종료
end_time = time.time()
print("총 소요 시간: %.3f초" %(end_time - start_time))

총 소요 시간: 91.306초
